# 非确定性运算---规约运算

Author: kaiyuan

Email: kyxie@zju.edu.cn



# 1 浮点数的结合律

在计算机系统中，浮点数运算并不严格遵循加法结合律， 可能出现：

(a + b) + c  != a + (b + c)

示例：


In [ ]:
# (a + b) + c  != a + (b + c) 浮点数计算示例：

import numpy as np

# float32能精确表示的最大连续整数：2^24=16777216
a = np.float32(16777216.0)
b = np.float32(1.0)
c = np.float32(-16777216.0)

# 强制存储结果，避免硬件扩展精度干扰
left = np.float32((a + b) + c)
right = np.float32(a + (b + c))

print("(a+b)+c =", left)   # 输出 0.0
print("a+(b+c) =", right)  # 输出 1.0
print("是否相等:", left == right)  # 输出 False

## 2 Triton中线程块对规约运算的影响
思路：通过Trition来演示非确定性运算，调整线程块BLOCK_SIZE来对比计算差异。

* 算子：规约(求和)运算。
* 参考系：pytorch自带的加法运算
* 数据类型：FP16、FP3

依赖包 torch、triton，建议在镜像环境下运行。

推荐：nvcr.io/nvidia/pytorch

## 2.1 运行代码


In [ ]:
import torch
import triton
import triton.language as tl


@triton.jit
def reduction_kernel_simple(
        x_ptr,  # 输入指针
        output_ptr,  # 输出指针
        n_elements,  # 元素数量
        BLOCK_SIZE: tl.constexpr,  # 线程块大小
):
    """简单的归约核函数"""
    pid = tl.program_id(axis=0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)

    # 加载数据
    mask = offsets < n_elements
    x = tl.load(x_ptr + offsets, mask=mask, other=0.0)

    # 简单的归约
    accumulator = tl.sum(x)

    # 存储结果
    tl.store(output_ptr + pid, accumulator)


def demonstrate_nondeterministic_triton():
    """演示Triton的非确定性计算"""

    print("=" * 80)
    print("Triton非确定性计算演示")
    print("使用不同线程块大小和内存访问模式")
    print("=" * 80)

    # 检查设备
    if not torch.cuda.is_available():
        print("警告: 未检测到CUDA设备")
        return

    device = torch.device('cuda')
    print(f"使用设备: {device}")

    # 设置随机种子
    torch.manual_seed(42)


    # 测试1: 不同线程块大小的向量归约
    print("\n1. 测试不同线程块大小的向量归约")
    print("-" * 40)

    # 创建数据
    vec_size = 2 ** 20  # 1M元素
    vector_fp16 = torch.randn(vec_size, dtype=torch.float16, device=device)
    vector_fp32 = vector_fp16.float()

    print(f"向量大小: {vec_size:,}")
    print(f"数据范围: [{vector_fp16.min().item():.3f}, {vector_fp16.max().item():.3f}]")

    # PyTorch基准
    torch_sum_fp16 = vector_fp16.sum().item()
    torch_sum_fp32 = vector_fp32.sum().item()
    print(f"\nPyTorch sum (float16): {torch_sum_fp16:.10f}")
    print(f"PyTorch sum (float32): {torch_sum_fp32:.10f}")
    print(f"两者差异: {abs(torch_sum_fp16 - torch_sum_fp32):.10f}")

    # 不同线程块大小
    block_sizes = [64, 128, 256, 512, 1024]
    results_fp16 = []
    results_fp32 = []

    for block_size in block_sizes:
        # 计算网格大小
        num_blocks = (vec_size + block_size - 1) // block_size
        grid = (num_blocks,)

        # 创建输出张量
        output_fp16 = torch.zeros(num_blocks, dtype=torch.float32, device=device)
        output_fp32 = torch.zeros(num_blocks, dtype=torch.float32, device=device)

        # 运行核函数
        reduction_kernel_simple[grid](
            vector_fp16, output_fp16, vec_size, BLOCK_SIZE=block_size
        )

        reduction_kernel_simple[grid](
            vector_fp32, output_fp32, vec_size, BLOCK_SIZE=block_size
        )

        # 计算最终结果
        result_fp16 = output_fp16.sum().item()
        result_fp32 = output_fp32.sum().item()

        results_fp16.append(result_fp16)
        results_fp32.append(result_fp32)

        print(f"\n线程块大小 {block_size}:")
        print(f"  float16结果: {result_fp16:.10f}")
        print(f"  与PyTorch差异: {abs(result_fp16 - torch_sum_fp16):.10f}")
        print(f"  float32结果: {result_fp32:.10f}")
        print(f"  与PyTorch差异: {abs(result_fp32 - torch_sum_fp32):.10f}")

    # 分析结果
    print("\n" + "-" * 40)
    print("结果分析:")
    max_diff_fp16 = max(results_fp16) - min(results_fp16)
    max_diff_fp32 = max(results_fp32) - min(results_fp32)

    print(f"float16不同线程块间的最大差异: {max_diff_fp16:.10f}")
    print(f"float32不同线程块间的最大差异: {max_diff_fp32:.10f}")

    if max_diff_fp32 > 0:
        print(f"float16差异是float32的 {max_diff_fp16 / max_diff_fp32:.2f} 倍")



if __name__ == "__main__":
    # 检查Triton是否可用
    try:
        import triton

        print(f"Triton版本: {triton.__version__}")
        # 运行演示
        demonstrate_nondeterministic_triton()

    except ImportError:
        print("错误: Triton未安装")
        print("请使用以下命令安装: pip install triton")
    except Exception as e:
        print(f"运行出错: {e}")
        import traceback

        traceback.print_exc()
    print("\n演示完成!")


## 2.1 输出

运行信息：

* GPU：NVIDIA A100-SXM4-80GB
* 固件：Driver Version: 570.172.08     CUDA Version: 12.8
* 镜像：nvcr.io/nvidia/pytorch:24.09-py3

```
Triton版本: 3.3.0
================================================================================
Triton非确定性计算演示
使用不同线程块大小和内存访问模式
================================================================================
使用设备: cuda

1. 测试不同线程块大小的向量归约
----------------------------------------
向量大小: 1,048,576
数据范围: [-4.594, 4.797]

PyTorch sum (float16): -143.8750000000
PyTorch sum (float32): -143.8523254395
两者差异: 0.0226745605

线程块大小 64:
  float16结果: -143.8328857422
  与PyTorch差异: 0.0421142578
  float32结果: -143.8524475098
  与PyTorch差异: 0.0001220703

线程块大小 128:
  float16结果: -143.6943359375
  与PyTorch差异: 0.1806640625
  float32结果: -143.8524169922
  与PyTorch差异: 0.0000915527

线程块大小 256:
  float16结果: -144.1240234375
  与PyTorch差异: 0.2490234375
  float32结果: -143.8523254395
  与PyTorch差异: 0.0000000000

线程块大小 512:
  float16结果: -144.4716796875
  与PyTorch差异: 0.5966796875
  float32结果: -143.8524780273
  与PyTorch差异: 0.0001525879

线程块大小 1024:
  float16结果: -144.1484375000
  与PyTorch差异: 0.2734375000
  float32结果: -143.8524169922
  与PyTorch差异: 0.0000915527

----------------------------------------
结果分析:
float16不同线程块间的最大差异: 0.7773437500
float32不同线程块间的最大差异: 0.0001525879
float16差异是float32的 5094.40 倍

演示完成!
```


## 附：矩阵乘法的比较

In [ ]:
@triton.jit
def matrix_mult_kernel_simple(
        a_ptr, b_ptr, c_ptr,
        M, N, K,
        stride_am, stride_ak,
        stride_bk, stride_bn,
        stride_cm, stride_cn,
        BLOCK_SIZE_M: tl.constexpr,
        BLOCK_SIZE_N: tl.constexpr,
        BLOCK_SIZE_K: tl.constexpr,
):
    """简单的矩阵乘法核函数"""
    pid_m = tl.program_id(axis=0)
    pid_n = tl.program_id(axis=1)

    # 计算偏移
    offs_am = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_bn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offs_k = tl.arange(0, BLOCK_SIZE_K)

    # 计算指针
    a_ptrs = a_ptr + (offs_am[:, None] * stride_am + offs_k[None, :] * stride_ak)
    b_ptrs = b_ptr + (offs_k[:, None] * stride_bk + offs_bn[None, :] * stride_bn)

    # 累加器
    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)

    # 矩阵乘法
    for k in range(0, K, BLOCK_SIZE_K):
        a = tl.load(a_ptrs, mask=offs_k[None, :] < K - k, other=0.0)
        b = tl.load(b_ptrs, mask=offs_k[:, None] < K - k, other=0.0)
        accumulator += tl.dot(a, b)
        a_ptrs += BLOCK_SIZE_K * stride_ak
        b_ptrs += BLOCK_SIZE_K * stride_bk

    # 存储结果
    offs_cm = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_cn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    c_ptrs = c_ptr + stride_cm * offs_cm[:, None] + stride_cn * offs_cn[None, :]
    mask = (offs_cm[:, None] < M) & (offs_cn[None, :] < N)
    tl.store(c_ptrs, accumulator, mask=mask)

In [ ]:
def matrix_compare():
    device = torch.device('cuda')
    print(f"使用设备: {device}")

    # 设置随机种子
    torch.manual_seed(42)

    # 禁用确定性算法
    torch.use_deterministic_algorithms(False)

    # 创建矩阵数据
    mat_size = 256  # 减小矩阵大小以加快计算
    A_fp16 = torch.randn((mat_size, mat_size), dtype=torch.float16, device=device)
    B_fp16 = torch.randn((mat_size, mat_size), dtype=torch.float16, device=device)
    A_fp32 = A_fp16.float()
    B_fp32 = B_fp16.float()

    print(f"矩阵大小: {mat_size}x{mat_size}")

    # PyTorch基准
    torch_result_fp16 = torch.matmul(A_fp16, B_fp16)
    torch_result_fp32 = torch.matmul(A_fp32, B_fp32)

    target_element = (50, 50)  # 选择一个目标元素
    torch_val_fp16 = torch_result_fp16[target_element].item()
    torch_val_fp32 = torch_result_fp32[target_element].item()

    print(f"PyTorch结果 (float16): {torch_val_fp16:.10f}")
    print(f"PyTorch结果 (float32): {torch_val_fp32:.10f}")

    # 不同图块大小的Triton矩阵乘法
    tile_configs = [
        {"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 32},
        {"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32},
        {"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 128, "BLOCK_SIZE_K": 32},
    ]

    matmul_results_fp16 = []
    matmul_results_fp32 = []

    for config in tile_configs:
        # 准备输出张量
        C_fp16 = torch.empty((mat_size, mat_size), dtype=torch.float16, device=device)
        C_fp32 = torch.empty((mat_size, mat_size), dtype=torch.float32, device=device)

        # 计算网格大小
        grid_m = triton.cdiv(mat_size, config["BLOCK_SIZE_M"])
        grid_n = triton.cdiv(mat_size, config["BLOCK_SIZE_N"])
        grid = (grid_m, grid_n)

        # 运行核函数
        matrix_mult_kernel_simple[grid](
            A_fp16, B_fp16, C_fp16,
            mat_size, mat_size, mat_size,
            A_fp16.stride(0), A_fp16.stride(1),
            B_fp16.stride(0), B_fp16.stride(1),
            C_fp16.stride(0), C_fp16.stride(1),
            BLOCK_SIZE_M=config["BLOCK_SIZE_M"],
            BLOCK_SIZE_N=config["BLOCK_SIZE_N"],
            BLOCK_SIZE_K=config["BLOCK_SIZE_K"],
        )

        matrix_mult_kernel_simple[grid](
            A_fp32, B_fp32, C_fp32,
            mat_size, mat_size, mat_size,
            A_fp32.stride(0), A_fp32.stride(1),
            B_fp32.stride(0), B_fp32.stride(1),
            C_fp32.stride(0), C_fp32.stride(1),
            BLOCK_SIZE_M=config["BLOCK_SIZE_M"],
            BLOCK_SIZE_N=config["BLOCK_SIZE_N"],
            BLOCK_SIZE_K=config["BLOCK_SIZE_K"],
        )

        # 获取结果
        val_fp16 = C_fp16[target_element].item()
        val_fp32 = C_fp32[target_element].item()

        matmul_results_fp16.append(val_fp16)
        matmul_results_fp32.append(val_fp32)

        print(f"\n图块配置 {config}:")
        print(f"  float16: {val_fp16:.10f}, 差异: {abs(val_fp16 - torch_val_fp16):.10f}")
        print(f"  float32: {val_fp32:.10f}, 差异: {abs(val_fp32 - torch_val_fp32):.10f}")


matrix_compare()